<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

In [1]:
!export HF_HOME=../hf_cache

In [2]:
!git config --global credential.helper store
!pip install accelerate
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

In [3]:
# import wandb
# from wandb import AlertLevel
# wandb.init(project="biollama_v2", # the project I am working on
#            tags=["hf_sft", "BioLlamaV2"]) # the Hyperparameters I want to keep track of

# wandb.alert(
#     title="Initialising training run",
#     text=f"We have started training",
#     level=AlertLevel.WARN,
#     wait_duration=300,
# )

In [4]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")
print(transformers.__version__)

/opt/miniconda3/envs/biollama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imported...
4.39.3


Next, we pull the huggingface token from the local config. Remember to make a copy of config_default.yml, name it config.yml and add your huggingface token. config.yml is in the gitignore so unless you misspell config.yml, it should not get pushed to github.

In [5]:
# token = input(f"Enter token: ")
from box import Box
with open("config.yml", "r") as f:
    config = Box.from_yaml(f.read())
token = config.secrets.hf_token

We also need to verfiy whether the GPU supports bfloat16 (less precision but more range). If not (for example if running on TITAN GPUs), we use float16

In [6]:
if torch.cuda.is_bf16_supported():
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16
torch_dtype = torch.bfloat16
print(f"Using dtype: {torch_dtype}")

Using dtype: torch.bfloat16


In [7]:
# from transformers import AutoModel
# medcpt_path = "ncbi/MedCPT-Article-Encoder"
# document_tokenizer = AutoTokenizer.from_pretrained(medcpt_path)
# document_model = AutoModel.from_pretrained(medcpt_path).to('cuda')

In [8]:
print("started script")

# llama_path = "h2oai/h2ogpt-4096-llama2-7b-chat"
llama_path = "meta-llama/Llama-2-70b-chat-hf"
# llama_path = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(llama_path, 
                                          token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained(llama_path, 
                                             token = token,
                                             device_map = "auto",
                                            # device_map = "cuda:1",
                                             torch_dtype = torch_dtype,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script


Loading checkpoint shards:   0%|                                                                                                                                                                                | 0/15 [00:00<?, ?it/s]

Loading checkpoint shards:   7%|███████████▏                                                                                                                                                            | 1/15 [01:27<20:25, 87.56s/it]

Loading checkpoint shards:  13%|██████████████████████▍                                                                                                                                                 | 2/15 [02:55<19:00, 87.76s/it]

Loading checkpoint shards:  20%|█████████████████████████████████▌                                                                                                                                      | 3/15 [04:23<17:34, 87.84s/it]

...

Loading checkpoint shards:  87%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                      | 13/15 [19:09<02:55, 87.97s/it]

Loading checkpoint shards:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 14/15 [20:33<01:26, 86.76s/it]

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [20:38<00:00, 62.18s/it]

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [20:38<00:00, 82.58s/it]

model loaded!


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
from time import time
max_new_tokens = 150
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda:1')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/pubmed_cleaned/"
print(path)
files = os.listdir(path)
print(files[:5])
# load in the first file and print first 20 lines
# with open(path + files[0], "r") as f:
#     lines = f.readlines()
#     for line in lines[:20]:
#         print(line)
from datasets import load_dataset
print(path + files[0])
dataset = load_dataset("text", 
                       cache_dir = "../hf_cache/",
                       data_files = path + files[0], 
                       split = "train")
# dataset = load_dataset("imdb", split="train")

In [ ]:
dataset

In [ ]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

for param in model.parameters(): 
    param.requires_grad = True
model.model.embed_tokens.weight.requires_grad_(False)
params, trainable_params = param_count(model)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
batch_size = 2
total_num_steps = 500000
output_dir = "../"
training_args = TrainingArguments(
    output_dir=output_dir,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size//2,
    # bf16=False,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=total_num_steps // 10,
    num_train_epochs=1,
    # max_steps = -1
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    # evaluation_strategy="steps",
    # eval_steps=total_num_steps // 6,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch", #changed to epoch so we save every epoch i guess?
    save_total_limit=1,
)

In [ ]:
def create_prompt(row):
    text = row["text"]
    return text
# create_prompt(dataset[0])

In [ ]:
train_dataset = dataset["text"]

In [ ]:
# print(train_dataset[0])

In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    # eval_dataset=test_dataset,
    packing=True,
    max_seq_length=512,
    args=training_args,
    formatting_func=create_prompt,
    # compute_metrics=token_accuracy,
)

In [ ]:
#set llama model config use_cache to false!!!
trainer.train()
# wandb.finish()

In [ ]:
# trainer.save_model(output_dir)
# #print contents of output_dir
# !ls -l $output_dir
# #print full path of output_dir
# # !pwd $output_dir

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class TextDataset(Dataset):
#     def __init__(self, data, tokenizer):
#         self.data = data
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         text = self.data[idx]['text']
#         inputs = self.tokenizer(text, 
#                                 return_tensors="pt", 
#                                 padding=True, 
#                                 max_length=10,
#                                 truncation=True)
#         return inputs
# text_dataset = TextDataset(dataset, tokenizer)
# dataloader = DataLoader(text_dataset, batch_size=1, shuffle=True)

In [ ]:
# import torch.optim as optim

# def train(model, dataloader, epochs=5, lr=5e-5):
#     device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#     # model.to(device)

#     optimizer = optim.AdamW(model.parameters(), lr=lr)
#     print("starting training")

#     for epoch in range(epochs):
#         model.train()
#         for item in dataset:
#             print(len(item))
#             text = item["text"]
#             inputs = tokenizer(text, return_tensors="pt", padding=True, max_length=10, truncation=True)
#             # print(batch)
#             inputs = inputs.to(device)
#             print(inputs.input_ids.shape)
#             print(inputs)
#             outputs = model(**inputs, labels=inputs.input_ids)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [ ]:
# dataset[0]["text"]

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token
# train(model, "dataloader", epochs=5, lr=5e-5)